**About** : This notebook is used to infer models.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
cd ../src/

/home/tviel/work/kaggle_birdclef_2024/src


/home/tviel/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
import os
import torch

print(torch.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
device = torch.cuda.get_device_name(0)
print(device)

2.2.0+cu121
NVIDIA RTX 6000 Ada Generation


### Imports

In [5]:
import os
import gc
import sys
import cv2
import glob
import json
import torch
import shutil
import joblib
import librosa
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn.functional as F
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from scipy.special import expit
from joblib import Parallel, delayed

warnings.simplefilter(action="ignore", category=UserWarning)

In [6]:
from util.logger import Config
from util.metrics import macro_auc
from util.torch import load_model_weights

from data.preparation import prepare_data, prepare_folds
from model_zoo.models import define_model
from inference.predict import infer_onnx, load_sample, infer_sample
from params import CLASSES

### Params

In [7]:
EVAL = False

In [8]:
if EVAL:
    DATA_PATH = "../input/train_audio/"
else:
    DATA_PATH = "../input/unlabeled_soundscapes/"

In [9]:
BATCH_SIZE = 48
USE_FP16 = True
NUM_WORKERS = 4

DEVICE = "cuda"

DURATION = 5
SR = 32000

In [10]:
FOLD = 0 if EVAL else "fullfit_0"

EXP_FOLDERS = [
    # ("../logs/2024-04-12/8/", [FOLD]),   # LB 0.64 baseline
    # ("../logs/2024-04-18/12/", [FOLD]),  #
    # ("../logs/2024-04-18/15/", [FOLD]),  #
    # ("../logs/2024-04-19/4/", [FOLD]),  # Change norm, sampling
    # ("../logs/2024-04-19/5/", [FOLD]),  # d=15s
    # ("../logs/2024-04-19/7/", [FOLD]),  # minmaxnorm, sampling, nocall, less mix
    # ("../logs/2024-04-19/8/", [FOLD]),  # minmaxnorm, no sampling, nocall, less mix no add
    # ("../logs/2024-04-19/10/", [FOLD]),  # minmaxnorm more mix more aug
    # ("../logs/2024-04-29/2/", [FOLD]),  # minmaxnorm fixed crop
    # ("../logs/2024-04-29/4/", [FOLD]),  # minmaxnorm fixed crop 20s selfmix
    # ("../logs/2024-04-29/6/", [FOLD]),  # minmaxnorm fixed crop no_xc selfmix
    # ("../logs/2024-04-29/7/", [FOLD]),  # minmaxnorm fixed crop no_xc selfmix focal_bce ousmk
    # ("../logs/2024-04-30/0/", [FOLD]),  # minmaxnorm selfmix focal_bce ousmk
    # ("../logs/2024-04-30/1/", [FOLD]),  # minmaxnorm selfmix focal_bce ousmk++
    # ("../logs/2024-04-30/3/", [FOLD]),  # minmaxnorm selfmix focal_bce no_xc more mix
    # ("../logs/2024-04-30/4/", [FOLD]),  # minmaxnorm selfmix bce no_xc ousmk^
    # ("../logs/2024-05-02/0/", [FOLD]),  # selfmix focal_bce ousmk + dedup, new melspec params, start-end sampling
    # ("../logs/2024-05-02/15/", [FOLD]),  # no selfmix focal_bce ousmk + 2nd mask, new melspec params, start-end sampling
    # ("../logs/2024-05-04/4/", [FOLD]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + 20 epochs, wd AdamW
    # ("../logs/2024-05-04/8/", [FOLD]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + 30 epochs, wd AdamW, upsample
    # ("../logs/2024-05-04/9/", [FOLD]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + 30 epochs, wd AdamW, upsample less augs
    # ("../logs/2024-05-05/1/", [FOLD]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + 20 epochs, wd AdamW new miw
    # ("../logs/2024-05-05/7/", [FOLD], "torch"),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + upsample no ext cls w
    # ("../logs/2024-05-06/1/", [FOLD]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + upsample no ext fix mix
    # ("../logs/2024-05-06/2/", [FOLD]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + upsample no ext fix mix b0
    # ("../logs/2024-05-06/3/", [FOLD]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + upsample no ext fix mix vit-b0
    # ("../logs/2024-05-06/12/", [0]),  # no selfmix bce + 2nd mask, new melspec params, start-end sampling + upsample no ext fix mix vit-b0
    # ("../logs/2024-05-06/18/", [FOLD], "torch"),  # vit-b0 bce ousmk + 2nd mask, new melspec params, start-end sampling + upsample no ext fix mix 3ch 
    # ("../logs/2024-05-06/19/", [FOLD], "torch"),  # v2s bce ousmk + 2nd mask, new melspec params, start-end sampling + upsample no ext fix mix 3ch 
    # ("../logs/2024-05-06/20/", [FOLD], "torch"),  # v2s bce ousmk + 2nd mask, new melspec params, start-end sampling + upsample no ext fix mix 3ch 
    # ("../logs/2024-05-06/22/", [FOLD], "torch"),  # vit-b1
    # ("../logs/2024-05-07/0/", [FOLD], "torch"),  # vit-b0
    # ("../logs/2024-05-07/1/", [f"fullfit_{i}" for i in range(4)], "torch"),  # vit-b0 bs32
    # ("../logs/2024-05-07/4/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b0 bs64
    # ("../logs/2024-05-07/5/", [f"{i}" for i in range(4)], "torch"),  # vit-b0 repro 0.64
    # ("../logs/2024-05-07/6/", [f"{i}" for i in range(4)], "torch"),  # v2-b0 bs64
    # ("../logs/2024-05-07/7/", [f"{i}" for i in range(4)], "torch"),  # vit-b1 bs64
    # ("../logs/2024-05-10/18/", [f"fullfit_{i}" for i in range(4, 5)], "torch"),  # vit-b0 PL 
    # ("../logs/2024-05-11/10/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b0 PL2
    # ("../logs/2024-05-11/11/", [f"fullfit_{i}" for i in range(5)], "torch"),  # effnet-b0 PL2
    # ("../logs/2024-05-12/0/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b0 PL0.72
    # ("../logs/2024-05-12/1/", [f"fullfit_{i}" for i in range(5)], "torch"),  # effnet-b0 PL0.72
    # ("../logs/2024-05-12/2/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b1 PL0.72
    # ("../logs/2024-05-13/0/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b0 PLBirdnet
    # ("../logs/2024-05-13/3/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b0 PL0.72 cpmp params
    # ("../logs/2024-05-14/0/", [f"fullfit_{i}" for i in range(5)], "torch"),  # vit-b0 PL0.72 cpmp params default head  <- LB 0.7

    # ("../logs/2024-05-14/17/", [f"{i}" for i in range(4)], "torch"),  # tinynet
    # ("../logs/2024-05-14/16/", [f"{i}" for i in range(4)], "torch"),  # mnasnet
    # ("../logs/2024-05-14/15/", [f"{i}" for i in range(4)], "torch"),  # mobilenet
    # ("../logs/2024-05-14/14/", [f"{i}" for i in range(4)], "torch"),  # mixnet
    # ("../logs/2024-05-14/12/", [f"{i}" for i in range(4)], "torch"),  # b0
    # ("../logs/2024-05-14/8/", [f"{i}" for i in range(4)], "torch"),  # b0-v2
    ("../logs/2024-05-14/18/", [f"{i}" for i in range(4)], "torch"),  # vit-b0
    # ("../logs/2024-05-14/19/", [f"{i}" for i in range(4)], "torch"),  # vit-b1
]

In [11]:
# from util.logger import upload_to_kaggle

# upload_to_kaggle(
#     [f[0] for f in EXP_FOLDERS],
#     directory="../output/dataset_4/",
#     dataset_name="BirdCLEF 2024 Weights 4",
#     update_folders=True
# )

### Preprocessing

In [12]:
if EVAL:
    df = pd.DataFrame({"path": glob.glob(DATA_PATH + "*/*")})
    df["id"] = df["path"].apply(lambda x: x.split("/")[-1][:-4])

    folds = pd.read_csv('../input/folds_4.csv')
    folds['id'] = folds['filename'].apply(lambda x: x.split('/')[-1][:-4])
    df = df.merge(folds)
    df = df[df['fold'] == 0].reset_index(drop=True)

    df["primary_label"] = df["path"].apply(lambda x:  x.split('/')[-2])
else:
    df = pd.DataFrame({"path": glob.glob(DATA_PATH + "*")})
    df["id"] = df["path"].apply(lambda x: x.split("/")[-1][:-4])
    
    # df["duration"] = df["path"].apply(lambda x: librosa.get_duration(path=x))
    # df = df[df["duration"] == 240].reset_index(drop=True)

### Models

In [13]:
models = []
for e in EXP_FOLDERS:
    try:
        exp_folder, folds, runtime = e
    except:
        exp_folder, folds = e
        runtime = "torch"
    
    config = Config(json.load(open(exp_folder + "config.json", "r")))

    for fold in folds:
        weights = exp_folder + f"{config.name}_{fold}.pt"

        model = define_model(
            config.name,
            config.melspec_config,
            head=config.head,
            aug_config=config.aug_config,
            num_classes=config.num_classes,
            n_channels=config.n_channels,
            drop_rate=config.drop_rate,
            drop_path_rate=config.drop_path_rate,
            norm=config.norm if hasattr(config, "norm") else "min_max",
            top_db=config.top_db if hasattr(config, "top_db") else None,
            exportable=config.exportable,
            verbose=True,
            pretrained=False
        )
        model = model.to(DEVICE).eval()

        model = load_model_weights(model, weights, verbose=config.local_rank == 0)
        models.append((model, runtime))


 -> Loading encoder weights from ../logs/2024-05-14/18/efficientvit_b0_0.pt


 -> Loading encoder weights from ../logs/2024-05-14/18/efficientvit_b0_1.pt


 -> Loading encoder weights from ../logs/2024-05-14/18/efficientvit_b0_2.pt


 -> Loading encoder weights from ../logs/2024-05-14/18/efficientvit_b0_3.pt



### Export

In [14]:
sessions = [None for _ in range(len(models))]
if any([runtime != "torch" for _, runtime in models]):
    sessions = []

    import onnx
    import onnxruntime as ort
    from onnxconverter_common import float16

    input_names = ['x']
    output_names = ['output']

    input_tensor = torch.randn(
        1 if EVAL else BATCH_SIZE,
        config.n_channels,
        config.melspec_config['n_mels'],
        313 if config.melspec_config['hop_length'] == 512 else 224
    )

    for i, (model, runtime) in enumerate(models):
        name = f"model_{i}.onnx"
        torch.onnx.export(
            model.encoder.cpu(),
            input_tensor,
            name,
            verbose=False,
            input_names=input_names,
            output_names=output_names,
        )
        onnx_model = onnx.load(name)
        # onnx_model = float16.convert_float_to_float16(onnx_model)
        # onnx.save(onnx_model, f"model_{i}.onnx")
        onnx.checker.check_model(onnx_model)
        ort_session = ort.InferenceSession(f"model_{i}.onnx")

        if runtime == "onnx":
            sessions.append(ort_session)
            print(f'- Convert model {name} to onnx')

        elif runtime == "openvino":
            import openvino.runtime as ov

            !mo --input_model $name # --compress_to_fp16=False
            
            core = ov.Core()
            openvino_model = core.read_model(model='model_0.xml')
            compiled_model = core.compile_model(openvino_model, device_name="CPU")
            infer_request = compiled_model.create_infer_request()
            sessions.append(infer_request)

            print(f'- Convert model {name} to openvino')
        else:
            sessions.append(None)

### Main

In [15]:
try:
    batches = np.array_split(np.arange(len(df)), len(df) / 100)
except:
    batches = [np.arange(len(df))]

In [16]:
inference_rows = {i : [] for i in range(len(models))}

for i, batch in enumerate(tqdm(batches)):
    df_batch = df.iloc[batch].reset_index(drop=True)

    waves = joblib.Parallel(n_jobs=4)(
        joblib.delayed(load_sample)(
            path, evaluate=EVAL, sr=SR, duration=DURATION, normalize=config.wav_norm
        )
        for path in df_batch["path"].values
    )

    for model_idx in range(len(models)):
        all_preds = [
            infer_sample(
                wave,
                [models[model_idx]],
                sessions,
                device=DEVICE,
                use_fp16=USE_FP16,
            )
            for wave in waves
        ]

        for idx in range(len(df_batch)):
            y_pred = all_preds[idx]
            preds = expit(y_pred).mean(0)

            for t, pred in enumerate(preds):
                predictions = dict([(l, p) for l, p in zip(CLASSES, pred)])
                inference_rows[model_idx].append(
                    {"row_id": f"{df_batch.id[idx]}_{(t + 1) * 5}"} | predictions
                )

        del all_preds
        gc.collect()
    
    del waves
    gc.collect()

    # break

for model_idx in range(len(models)):
    sub = pd.DataFrame(inference_rows[model_idx])
    if not EVAL:
        name = EXP_FOLDERS[0][0] + f'pl_preds_{EXP_FOLDERS[0][1][model_idx]}'
        sub[["row_id"]].to_csv(name + ".csv", index=False)
        np.save(name + ".npy", sub[CLASSES].values)

        print(f"-> Saved predictions to {name}[.csv/.npy]")
        display(sub.head(2))

    del (sub, inference_rows[model_idx])
    gc.collect()

  0%|          | 0/84 [00:00<?, ?it/s]

-> Saved predictions to ../logs/2024-05-14/18/pl_sub_0


,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,184575141_5,0.004332,0.014504,0.005911,0.001240,0.001755,0.000804,0.000929,0.002252,0.007432,...,0.000333,0.000729,0.005220,0.010694,0.004944,0.060087,0.002332,0.001211,0.000197,0.020964
1,184575141_10,0.002406,0.002981,0.001629,0.000503,0.002278,0.000629,0.000636,0.000846,0.002415,...,0.000421,0.000503,0.004315,0.008577,0.004166,0.029817,0.001942,0.000651,0.000301,0.050051


-> Saved predictions to ../logs/2024-05-14/18/pl_sub_1


,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,184575141_5,0.002387,0.010130,0.001082,0.000667,0.001501,0.000933,0.001294,0.008125,0.003497,...,0.000406,0.000551,0.001289,0.007289,0.027954,0.051750,0.002632,0.002149,0.000497,0.005820
1,184575141_10,0.007403,0.002704,0.001134,0.000497,0.011464,0.002715,0.001472,0.001996,0.004849,...,0.001782,0.000521,0.002715,0.003470,0.004521,0.012384,0.002360,0.002561,0.000859,0.025663


-> Saved predictions to ../logs/2024-05-14/18/pl_sub_2


,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,184575141_5,0.003915,0.004248,0.001188,0.000386,0.001183,0.00131,0.001001,0.001065,0.003552,...,0.000634,0.000944,0.001875,0.010860,0.002958,0.046378,0.000804,0.001817,0.000936,0.009784
1,184575141_10,0.005060,0.002226,0.001284,0.000253,0.004248,0.00189,0.000883,0.000641,0.006004,...,0.002541,0.000830,0.003197,0.007121,0.003552,0.017177,0.000767,0.000699,0.000752,0.033717


-> Saved predictions to ../logs/2024-05-14/18/pl_sub_3


,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,184575141_5,0.003028,0.008711,0.002243,0.000820,0.000662,0.001721,0.000600,0.003552,0.001383,...,0.000358,0.001484,0.002141,0.003430,0.018055,0.043528,0.000680,0.002958,0.000144,0.007726
1,184575141_10,0.002946,0.002601,0.000792,0.000795,0.002601,0.002191,0.000501,0.000582,0.001264,...,0.001897,0.001040,0.003443,0.004646,0.006193,0.020646,0.000785,0.000843,0.000412,0.014064


In [ ]:
if EVAL:
    preds = sub[CLASSES].values
    auc = macro_auc(df["primary_label"].values.tolist()[:len(preds)], preds)
    print(f'Fold 0 AUC: {auc:.5f}')

Done !